# Crypto_Scraper

## Import 

In [14]:
import praw
import pandas as pd
from datetime import datetime
import os
import psaw as ps
from dotenv import load_dotenv
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F

In [15]:
# Lade die .env-Datei
dotenv_loaded = load_dotenv("zugang_reddit.env")  # Falls die Datei anders heißt, anpassen
# Prüfe, ob die Datei geladen wurde
print(f".env geladen? {dotenv_loaded}")


.env geladen? True


In [16]:
# Verbuindung zur Pushshift-API
reddit = praw.Reddit(
    client_id=os.getenv("CLIENT_ID"),
    client_secret=os.getenv("CLIENT_SECRET"),
    user_agent=os.getenv("USER_AGENT")
)

print("Reddit API erfolgreich verbunden!")


Reddit API erfolgreich verbunden!


In [17]:
# Test der Verbindung
try:
    subreddit = reddit.subreddit("CryptoCurrency")
    for post in subreddit.hot(limit=5):
        print(f"Title: {post.title}, Score: {post.score}")
except Exception as e:
    print(f"Fehler beim Abrufen der Subreddit-Daten: {e}")


Title: Kraken Pay AMA with Jared Blake, $5000 USDG* in prizes up for grabs!, Score: 32
Title: Daily Crypto Discussion - February 4, 2025 (GMT+0), Score: 43
Title: Musk has confirmed he wants to put the U.S. Treasury on a blockchain, Score: 3995
Title: Coinbase pushes US regulators to let banks offer crypto services once and for all, Score: 343
Title: Someone Just Moved 50 Bitcoin (BTC) Mined 15 Years Ago - HODL'ed from $0.10 to $100K. Who of You Just Got Out of Prison?, Score: 1746


## Liste der Cryptos

In [18]:
cryptos = [
    ("Ethereum", "ETH"),
    ("Solana", "SOL"),
    ("Avalanche", "AVAX"),
    ("Polkadot", "DOT"),
    ("Near Protocol", "NEAR"),
    ("Polygon", "MATIC"),
    ("XRP", "XRP"),
    ("Cardano", "ADA"),
    ("Chainlink", "LINK"),
    ("PYR", "PYR"),
    ("Cronos", "CRO"),
    ("Chiliz", "CHZ"),
    ("Illuvium", "ILV"),
    ("Ronin", "RON"),
    ("Band Protocol", "BAND"),
    ("Opulous", "OPUL"),
    ("Tia", "TIA"),
    ("Numeraire", "NMR"),
    ("Athens", "ATH"),
    ("Sui", "SUI"),  # Dein gewünschtes SUI-Token
    ("Binance Coin", "BNB"),
    ("Dogecoin", "DOGE"),
    ("Litecoin", "LTC"),
    ("Uniswap", "UNI"),
    ("Cosmos", "ATOM"),
    ("Aave", "AAVE"),
    ("The Graph", "GRT"),
    ("Decentraland", "MANA"),
    ("Sandbox", "SAND"),
    ("Axie Infinity", "AXS"),
    ("Render Token", "RNDR"),
    ("Fantom", "FTM"),
    ("Helium", "HNT"),
    ("Gala", "GALA"),
    ("Theta Network", "THETA"),
    ("Kava", "KAVA"),
    ("Stacks", "STX"),
    ("PancakeSwap", "CAKE"),
    ("1inch", "1INCH"),
    ("Enjin Coin", "ENJ"),
    ("Curve DAO", "CRV"),
    ("Convex Finance", "CVX"),
    ("Flow", "FLOW"),
    ("Arweave", "AR"),
    ("Celo", "CELO"),
    ("Immutable X", "IMX"),
    ("dYdX", "DYDX"),
    ("Lido DAO", "LDO"),
    ("Rocket Pool", "RPL"),
    ("Algorand", "ALGO"),
]


In [19]:
# Die Subreddit Liste
subreddits = [
    "CryptoCurrency",  # Allgemeine Diskussionen über Kryptowährungen
    "CryptoMarkets",   # Diskussionen über den Kryptomarkt und Preisbewegungen
    "CryptoTrading",   # Fokus auf Trading-Strategien und Analysen
    "Altcoin",         # Diskussionen über Altcoins (alle Kryptowährungen außer Bitcoin)
    "DeFi",            # Decentralized Finance (DeFi) und Projekte
    "BitcoinBeginners",# Für Anfänger in der Krypto-Welt
    "cryptotechnology", # Fokus auf die zugrunde liegende Blockchain-Technologie
    "cryptocurrencies", # Allgemeine Diskussionen über Kryptowährungen
    "Satoshistreetsbets", # Krypto-Wetten und Spekulationen
    "Binance"        # Diskussionen über die Binance-Plattform  
]

## Scrape

In [20]:
# Funktion, um Posts und Kommentare zu scrapen
def scrape_reddit(start_timestamp, end_timestamp, mode="initial"):
    posts = []
    comments = []
    post_ids = set()  # Set zum Speichern der Post-IDs

    for crypto_name, crypto_symbol in cryptos:
        for subreddit_name in subreddits:
            subreddit = reddit.subreddit(subreddit_name)
            print(f"Suche Posts zu {crypto_name} ({crypto_symbol}) in r/{subreddit_name}...")

            # Suche nach Namen und Kürzel
            for search_term in [crypto_name, crypto_symbol]:
                for post in subreddit.search(query=search_term, sort="new", limit=None):
                    if start_timestamp <= post.created_utc <= end_timestamp and post.id not in post_ids:
                        post_ids.add(post.id)
                        posts.append({
                            'crypto': crypto_name,
                            'search_term': search_term,
                            'subreddit': subreddit_name,
                            'post_id': post.id,
                            'title': post.title,
                            'author': str(post.author),
                            'created_utc': datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                            'score': post.score,
                            'num_comments': post.num_comments,
                            'selftext': post.selftext
                        })
                        print(f"Gefunden in r/{subreddit_name}: {post.title} (Suchbegriff: {search_term})")

                        # Kommentare sammeln
                        post.comments.replace_more(limit=0)
                        for comment in post.comments.list():
                            comments.append({
                                'post_id': post.id,
                                'comment_id': comment.id,
                                'author': str(comment.author),
                                'created_utc': datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                                'score': comment.score,
                                'body': comment.body
                            })

    # In DataFrames umwandeln
    df_posts = pd.DataFrame(posts)
    df_comments = pd.DataFrame(comments)

    # DataFrames zurückgeben
    print(f"Scrape abgeschlossen: {len(df_posts)} Posts und {len(df_comments)} Kommentare gefunden.")
    return df_posts, df_comments

# **Wöchentlicher Scrape**
now = int(datetime.now().timestamp())  # Aktueller Zeitstempel
last_week = now - 7 * 24 * 60 * 60  # 7 Tage zurück
print("Starte den wöchentlichen Scrape...")
df_posts_weekly, df_comments_weekly = scrape_reddit(last_week, now, mode="weekly")

# Beispiel: Lokale Weiterverarbeitung
print("Daten können jetzt in der Pipeline bereinigt werden...")

Starte den wöchentlichen Scrape...
Suche Posts zu Ethereum (ETH) in r/CryptoCurrency...
Gefunden in r/CryptoCurrency: Daily transaction cryptocurrency (Suchbegriff: Ethereum)
Gefunden in r/CryptoCurrency: Ethereum Co-Founder Lubin: Whales Are Manipulating ETH Price (Suchbegriff: Ethereum)
Gefunden in r/CryptoCurrency: The worst thing that happened to Ethereum' — Bitcoin up 160% since the Merge (Suchbegriff: Ethereum)
Gefunden in r/CryptoCurrency: Pumpdotfun's monthly revenue hit an all-time high of $121M in January, surpassing Ethereum’s monthly revenue for the first time. (Suchbegriff: Ethereum)
Gefunden in r/CryptoCurrency: Ethereum News: ETH in Focus as Gas Limits Raised for First Time Since 2021 (Suchbegriff: Ethereum)
Gefunden in r/CryptoCurrency: For every action there is an equal and opposite reaction. (Suchbegriff: Ethereum)
Gefunden in r/CryptoCurrency: Ethereum leads crypto liquidation spree with $617M losses in 24 hours (Suchbegriff: Ethereum)
Gefunden in r/CryptoCurrency: Q

## Datenbereinigung 

Fubktion zur Bereinigung der Daten 

In [21]:
def clean_data(df_posts, df_comments, comment_threshold=500):# Anpassbarer Schwellenwert für Kommentare pro Nutzer
    # 1. Duplikate entfernen
    df_posts = df_posts.drop_duplicates(subset=["post_id"])
    df_comments = df_comments.drop_duplicates(subset=["comment_id"])
    
    # 2. Fehlende Werte behandeln
    df_posts['selftext'] = df_posts['selftext'].fillna('')  # Fehlende Posttexte auffüllen
    df_comments['body'] = df_comments['body'].fillna('')  # Fehlende Kommentare auffüllen
    
    # 3. Zeitstempel konvertieren
    df_posts['created_utc'] = pd.to_datetime(df_posts['created_utc'])
    df_comments['created_utc'] = pd.to_datetime(df_comments['created_utc'])

    # 4. Datum & Uhrzeit in separate Spalten aufteilen (Daten normalisieren)
    df_posts["date"] = df_posts["created_utc"].dt.date  # YYYY-MM-DD
    df_posts["time"] = df_posts["created_utc"].dt.time  # HH:MM:SS

    df_comments["date"] = df_comments["created_utc"].dt.date
    df_comments["time"] = df_comments["created_utc"].dt.time

    # 5. Original `created_utc`-Spalte entfernen
    df_posts.drop(columns=["created_utc"], inplace=True)
    df_comments.drop(columns=["created_utc"], inplace=True)

    # 6. Filterung nach Qualität (Spam oder irrelevante Daten entfernen)
    df_posts = df_posts[df_posts['score'] > 0]  # Posts mit negativem Score entfernen
    df_comments = df_comments[df_comments['score'] > 0]  # Kommentare mit negativem Score entfernen

    # 7. Entferne bekannte Bot-Accounts
    bot_accounts = ["AutoModerator", "coinfeeds-bot", "devCheckingIn"]
    df_comments = df_comments[~df_comments["author"].isin(bot_accounts)]

    # 8. Entferne Nutzer mit übermäßigen Kommentaren
    comment_counts = df_comments["author"].value_counts()
    frequent_users = comment_counts[comment_counts > comment_threshold].index  # Nutzer über Grenze
    df_comments = df_comments[~df_comments["author"].isin(frequent_users)]

    print(f"✅ Daten bereinigt: {df_comments.shape[0]} Kommentare übrig (nach Spam-Filter).")

    return df_posts, df_comments


Funktion zur bereinigung auf die Daten anwenden

In [22]:
# Bereinigen der Daten
df_posts_clean, df_comments_clean = clean_data(df_posts_weekly, df_comments_weekly)

# Überprüfen, wie viele Einträge übrig sind
print(f"Bereinigte Posts: {len(df_posts_clean)}")
print(f"Bereinigte Kommentare: {len(df_comments_clean)}")

✅ Daten bereinigt: 6827 Kommentare übrig (nach Spam-Filter).
Bereinigte Posts: 136
Bereinigte Kommentare: 6827


## Model fuer das Sentiment

In [23]:
# Neues Modell laden
MODEL_NAME = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

# 🔹 Sicherstellen, dass die Spalte "body" existiert
if "body" not in df_comments_clean.columns:
    raise ValueError("Fehler: Die CSV-Datei enthält keine 'body'-Spalte mit Kommentaren!")

# 🔹 Funktion zur Sentiment-Analyse
def analyze_sentiment(text):
    if not isinstance(text, str) or text.strip() == "":
        return "neutral", 0.0  # Leere Kommentare sind neutral

    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)

    scores = F.softmax(outputs.logits, dim=1)[0]
    labels = ["negative", "neutral", "positive"]
    sentiment = labels[torch.argmax(scores).item()]
    confidence = scores.max().item()

    return sentiment, confidence

# 🔹 Sentiment für alle Kommentare berechnen
df_comments_clean["sentiment"], df_comments_clean["sentiment_confidence"] = zip(*df_comments_clean["body"].map(analyze_sentiment))

In [27]:
df_comments_clean.head()

,post_id,comment_id,author,score,body,date,time,sentiment,sentiment_confidence
0,1ihjtjy,maxv5r6,Snakeboard_OG,1,Kaspa is my go to. \n\nMy reasoning being- 10...,2025-02-04,16:01:31,positive,0.942727
1,1ihjtjy,maxv719,blaziken8x,1,I'd imagine anything that's fast and the fees ...,2025-02-04,16:01:43,neutral,0.675316
2,1ihjtjy,maxzxcg,Appropriate-Talk-735,1,I expect banks will offer solutions to spend y...,2025-02-04,16:24:17,neutral,0.806882
3,1ihjtjy,may2jxt,jawni,1,">So - what do you think is likely to be the ""d...",2025-02-04,16:36:38,neutral,0.590614
4,1ihjtjy,may2803,Donkey_Launcher,1,"They'll have to at some point, otherwise crypt...",2025-02-04,16:35:04,neutral,0.483234


## Google Drive Export

Pfad

In [24]:
# Definiere den Speicherpfad zu deinem lokalen Google Drive
DRIVE_PATH = "G:/Meine Ablage/reddit"

Export Funktion

In [30]:
def update_csv(df_new, filename, key_column):
    """Ersetzt bestehende Einträge basierend auf `key_column` und speichert die CSV sauber ab."""
    file_path = os.path.join(DRIVE_PATH, filename)

    try:
        # 🔹 Falls die Datei existiert, lade bestehende Daten
        if os.path.exists(file_path):
            df_existing = pd.read_csv(file_path, sep="|", encoding="utf-8-sig", on_bad_lines="skip")

            # 🔹 Neue Daten einfügen & sicherstellen, dass keine Duplikate entstehen
            df_combined = pd.concat([df_existing, df_new]).drop_duplicates(subset=[key_column], keep="last").reset_index(drop=True)
        else:
            df_combined = df_new  # Falls keine Datei existiert, einfach neue Daten speichern

        # 🔹 CSV speichern mit optimierter Formatierung
        df_combined.to_csv(
            file_path,
            index=False,
            sep="|",  # Einzigartiger Separator für Excel
            encoding="utf-8-sig",  # UTF-8 für Sonderzeichen
            lineterminator="\n"  # Richtige Zeilenumbrüche
        )
        print(f"✅ Datei erfolgreich aktualisiert: {file_path}")

    except Exception as e:
        print(f"⚠️ Fehler beim Speichern der Datei {filename}: {e}")

def export_to_drive(df_posts_clean, df_comments_clean):
    """Exportiert bereinigte Daten in Drive und ersetzt doppelte Einträge."""
    try:
        # 🔹 Posts aktualisieren (nach `post_id`)
        update_csv(df_posts_clean, "reddit_posts.csv", key_column="post_id")
        # 🔹 Kommentare aktualisieren (nach `comment_id`), ohne Dopplungen
        update_csv(df_comments_clean, "reddit_comments.csv", key_column="comment_id")

    except Exception as e:
        print(f"⚠️ Fehler beim Export: {e}")

Export Funktion anwenden

In [32]:
export_to_drive(df_posts_clean, df_comments_clean)

✅ Datei erfolgreich aktualisiert: G:/Meine Ablage/reddit\reddit_posts.csv
✅ Datei erfolgreich aktualisiert: G:/Meine Ablage/reddit\reddit_comments.csv
